In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [2]:
path='data/baselinetest_pred/labels/'
img='data/test'
out='submission.csv'

In [3]:
def rm_overlap(df,iou_threshold=0.8):
    for index1, row1 in df.iterrows():
        bb1,bb2={},{}
        bb1['x1'],bb1['y1']=row1['bb_left'],row1['bb_top']
        bb1['x2'],bb1['y2']=row1['bb_left']+row1['width'],row1['bb_top']+row1['height']
        for index2, row2 in df.iterrows():
            if index2==index1 or row1['class']!= row2['class']: continue
            bb2['x1'],bb2['y1']=row2['bb_left'],row2['bb_top']
            bb2['x2'],bb2['y2']=row2['bb_left']+row2['width'],row2['bb_top']+row2['height']
            if get_iou(bb1,bb2)>iou_threshold:
                df=df.drop(index2)
    return df
            
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [4]:
appended_data = []
for i in tqdm(sorted(os.listdir(path))):
    df=pd.read_csv(path+i,delimiter=' ',header=None)
    df.columns=['class', 'x_centre', 'y_centre', 'width', 'height']
    df['vid']=int(i[:3])
    df['frame_id']=int(i[4:-4])
    df['bb_left']=df['x_centre']-df['width']/2.0
    df['bb_top']=df['y_centre']-df['height']/2.0
    df['confidence']=1.0
    df['class']=df['class']+1
    df = df[['vid','frame_id','bb_left','bb_top', 'width', 'height','class','confidence']]
    df=rm_overlap(df,0.5)
    #debug
    img=i[:-3]+'png'
    
    break
    appended_data.append(df)
    
appended_data
    

100%|████████████████████████████████████| 17188/17188 [02:40<00:00, 107.31it/s]


[   vid  frame_id   bb_left    bb_top     width    height  class  confidence
 0    1         1  0.271875  0.319444  0.027344  0.090278      2         1.0
 1    1         1  0.271875  0.319445  0.027344  0.091667      3         1.0
 2    1         1  0.186718  0.393055  0.025781  0.070833      1         1.0
 3    1         1  0.186719  0.343056  0.025000  0.091667      2         1.0
 4    1         1  0.272656  0.365277  0.027344  0.073611      1         1.0,
    vid  frame_id   bb_left    bb_top     width    height  class  confidence
 0    1        10  0.311719  0.375000  0.031250  0.106944      3         1.0
 1    1        10  0.311719  0.373611  0.030469  0.105556      2         1.0
 2    1        10  0.217968  0.390278  0.029687  0.106944      2         1.0
 3    1        10  0.217187  0.434723  0.033594  0.091667      1         1.0
 4    1        10  0.311719  0.420833  0.032813  0.097222      1         1.0,
    vid  frame_id   bb_left    bb_top     width    height  class  confiden

In [7]:
appended_data = pd.concat(appended_data)
appended_data

,vid,frame_id,bb_left,bb_top,width,height,class,confidence
0,1,1,0.271875,0.319444,0.027344,0.090278,2,1.0
1,1,1,0.271875,0.319445,0.027344,0.091667,3,1.0
2,1,1,0.186718,0.393055,0.025781,0.070833,1,1.0
3,1,1,0.186719,0.343056,0.025000,0.091667,2,1.0
4,1,1,0.272656,0.365277,0.027344,0.073611,1,1.0
...,...,...,...,...,...,...,...,...
5,100,99,0.327344,0.344445,0.041406,0.134722,2,1.0
6,100,99,0.426562,0.381944,0.073438,0.105556,1,1.0
7,100,99,0.327343,0.434722,0.038281,0.084722,1,1.0
8,100,99,0.519532,0.737500,0.086719,0.244444,2,1.0


In [8]:
appended_data.to_csv(out)

# Visualize

In [ ]:
img_path=
for index, row in appended_data.iterrows():
    